In [1]:
import numpy as np
import pandas as pd
import sklearn
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import linear_kernel
from Data_cleaning import *

# Xử lý dữ liệu 

In [2]:
#import books, users, ratings clean data
books, users, ratings = get_clean_data(path="/content/drive/My Drive/Colab Notebooks/recommendation_system/data/")

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:28

In [3]:
books.head()

,isbn,title,author,pub_year,publisher,url_s,url_m,url_l
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
ratings.head()

,user,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
#number of ratings received by each book
usersPerIsbn = ratings['isbn'].value_counts()
usersPerIsbn

0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
              ... 
9728440030       1
0425022021       1
0263745082       1
3870700513       1
0836217233       1
Name: isbn, Length: 340556, dtype: int64

In [6]:
books_10 = books[books['isbn'].isin(usersPerIsbn[usersPerIsbn>10].index)]
books_10 = books_10.reset_index(drop=True)
# books_10 = books[:50000]

# Chỉ sử dụng `Book-Title`

In [7]:
vectorizer = TfidfVectorizer(analyzer='word')
#build book-title tfidf matrix
tfidf_matrix = vectorizer.fit_transform(books_10['title'])
tfidf_matrix.shape

(15452, 10972)

In [8]:
#Function to get the most similar books
indices_n = pd.Series(books_10['isbn']).to_dict()
inddict = dict((v,k) for k,v in indices_n.items())
def recommend_cosine(isbn, method):
    id = inddict[isbn]
    # sorting them and getting top 5
    similarity_scores = list(enumerate(method[id]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[:6]
    
    #Get the books index
    books_index = [i[0] for i in similarity_scores]
    
    #Return the top 5 most similar books using integar-location based indexing (iloc)
    return books_10.iloc[books_index]

def recommend_euclidean (isbn, method):
    id = inddict[isbn]
    # Get the pairwise similarity scores of all books compared that book,
    # sorting them and getting top 5
    similarity_scores = list(enumerate(method[id]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1])
    similarity_scores = similarity_scores[:6]

    #Get the books index
    books_index = [i[0] for i in similarity_scores]
    
    #Return the top 5 most similar books using integar-location based indexing (iloc)
    return books_10.iloc[books_index]

In [9]:
isbn_test = "0671723650"
books_10.iloc[inddict[isbn_test]]

isbn                                                0671723650
title                  How To Win Friends And Influence People
author                                           Dale Carnegie
pub_year                                                  1990
publisher                                               Pocket
url_s        http://images.amazon.com/images/P/0671723650.0...
url_m        http://images.amazon.com/images/P/0671723650.0...
url_l        http://images.amazon.com/images/P/0671723650.0...
Name: 2755, dtype: object

## Cosine similarity

In [10]:
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)


In [11]:
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)
recommend_cosine(isbn_test, cosine_similarity).head()

,isbn,title,author,pub_year,publisher,url_s,url_m,url_l
2755,0671723650,How To Win Friends And Influence People,Dale Carnegie,1990.0,Pocket,http://images.amazon.com/images/P/0671723650.0...,http://images.amazon.com/images/P/0671723650.0...,http://images.amazon.com/images/P/0671723650.0...
584,067142517X,"HOW TO WIN FRIENDS, REVISED",Dale Carnegie,1981.0,Simon &amp; Schuster,http://images.amazon.com/images/P/067142517X.0...,http://images.amazon.com/images/P/067142517X.0...,http://images.amazon.com/images/P/067142517X.0...
1579,030681188X,How to Lose Friends &amp; Alienate People,Toby Young,2002.0,DaCapo Press,http://images.amazon.com/images/P/030681188X.0...,http://images.amazon.com/images/P/030681188X.0...,http://images.amazon.com/images/P/030681188X.0...
12304,0306812274,How to Lose Friends &amp; Alienate People,Toby Young,2003.0,Da Capo Press,http://images.amazon.com/images/P/0306812274.0...,http://images.amazon.com/images/P/0306812274.0...,http://images.amazon.com/images/P/0306812274.0...
5657,0451209443,Sphere of Influence,Kyle Mills,2003.0,Signet Book,http://images.amazon.com/images/P/0451209443.0...,http://images.amazon.com/images/P/0451209443.0...,http://images.amazon.com/images/P/0451209443.0...


## Euclidean Distance

In [12]:
euclidean = euclidean_distances(tfidf_matrix)
recommend_euclidean(isbn_test, euclidean)


,isbn,title,author,pub_year,publisher,url_s,url_m,url_l
2755,0671723650,How To Win Friends And Influence People,Dale Carnegie,1990.0,Pocket,http://images.amazon.com/images/P/0671723650.0...,http://images.amazon.com/images/P/0671723650.0...,http://images.amazon.com/images/P/0671723650.0...
584,067142517X,"HOW TO WIN FRIENDS, REVISED",Dale Carnegie,1981.0,Simon &amp; Schuster,http://images.amazon.com/images/P/067142517X.0...,http://images.amazon.com/images/P/067142517X.0...,http://images.amazon.com/images/P/067142517X.0...
2050,0151010633,Q,Luther Blissett,2004.0,Harcourt,http://images.amazon.com/images/P/0151010633.0...,http://images.amazon.com/images/P/0151010633.0...,http://images.amazon.com/images/P/0151010633.0...
4208,0452281881,e,Matt Beaumont,2000.0,Plume Books,http://images.amazon.com/images/P/0452281881.0...,http://images.amazon.com/images/P/0452281881.0...,http://images.amazon.com/images/P/0452281881.0...
5043,0449216527,S.,John Updike,1994.0,Fawcett Books,http://images.amazon.com/images/P/0449216527.0...,http://images.amazon.com/images/P/0449216527.0...,http://images.amazon.com/images/P/0449216527.0...
9335,155166822X,T L C,Barbara Delinsky,2001.0,Mira,http://images.amazon.com/images/P/155166822X.0...,http://images.amazon.com/images/P/155166822X.0...,http://images.amazon.com/images/P/155166822X.0...


# Sử dụng 3 trường `Book-Title`, `Book-Author`, `Publisher`


In [14]:
books_10['all_content'] = books_10['title'] + books_10['author'] + books_10['publisher']
tfidf_all_content = vectorizer.fit_transform(books_10['all_content'])
tfidf_all_content.shape

(15452, 25806)

## Cosine similarity

In [15]:
# comping cosine similarity matrix using linear_kernal of sklearn
cosine_similarity_all_content = linear_kernel(tfidf_all_content, tfidf_all_content)
recommend_cosine(isbn_test, cosine_similarity_all_content)

,isbn,title,author,pub_year,publisher,url_s,url_m,url_l,all_content
2755,0671723650,How To Win Friends And Influence People,Dale Carnegie,1990.0,Pocket,http://images.amazon.com/images/P/0671723650.0...,http://images.amazon.com/images/P/0671723650.0...,http://images.amazon.com/images/P/0671723650.0...,How To Win Friends And Influence PeopleDale Ca...
584,067142517X,"HOW TO WIN FRIENDS, REVISED",Dale Carnegie,1981.0,Simon &amp; Schuster,http://images.amazon.com/images/P/067142517X.0...,http://images.amazon.com/images/P/067142517X.0...,http://images.amazon.com/images/P/067142517X.0...,"HOW TO WIN FRIENDS, REVISEDDale CarnegieSimon ..."
2756,0671733354,How to Stop Worrying and Start Living,Dale Carnegie,1990.0,Pocket,http://images.amazon.com/images/P/0671733354.0...,http://images.amazon.com/images/P/0671733354.0...,http://images.amazon.com/images/P/0671733354.0...,How to Stop Worrying and Start LivingDale Carn...
1579,030681188X,How to Lose Friends &amp; Alienate People,Toby Young,2002.0,DaCapo Press,http://images.amazon.com/images/P/030681188X.0...,http://images.amazon.com/images/P/030681188X.0...,http://images.amazon.com/images/P/030681188X.0...,How to Lose Friends &amp; Alienate PeopleToby ...
12304,0306812274,How to Lose Friends &amp; Alienate People,Toby Young,2003.0,Da Capo Press,http://images.amazon.com/images/P/0306812274.0...,http://images.amazon.com/images/P/0306812274.0...,http://images.amazon.com/images/P/0306812274.0...,How to Lose Friends &amp; Alienate PeopleToby ...
11885,0875420133,How to See and Read the Aura (Llewellyn's How ...,Ted Andrews,1991.0,Llewellyn Publications,http://images.amazon.com/images/P/0875420133.0...,http://images.amazon.com/images/P/0875420133.0...,http://images.amazon.com/images/P/0875420133.0...,How to See and Read the Aura (Llewellyn's How ...


---

## Euclidean Distance

In [16]:
euclidean_all_contend = euclidean_distances(tfidf_all_content)
recommend_euclidean(isbn_test, euclidean_all_contend)

,isbn,title,author,pub_year,publisher,url_s,url_m,url_l,all_content
2755,0671723650,How To Win Friends And Influence People,Dale Carnegie,1990.0,Pocket,http://images.amazon.com/images/P/0671723650.0...,http://images.amazon.com/images/P/0671723650.0...,http://images.amazon.com/images/P/0671723650.0...,How To Win Friends And Influence PeopleDale Ca...
584,067142517X,"HOW TO WIN FRIENDS, REVISED",Dale Carnegie,1981.0,Simon &amp; Schuster,http://images.amazon.com/images/P/067142517X.0...,http://images.amazon.com/images/P/067142517X.0...,http://images.amazon.com/images/P/067142517X.0...,"HOW TO WIN FRIENDS, REVISEDDale CarnegieSimon ..."
2756,0671733354,How to Stop Worrying and Start Living,Dale Carnegie,1990.0,Pocket,http://images.amazon.com/images/P/0671733354.0...,http://images.amazon.com/images/P/0671733354.0...,http://images.amazon.com/images/P/0671733354.0...,How to Stop Worrying and Start LivingDale Carn...
1579,030681188X,How to Lose Friends &amp; Alienate People,Toby Young,2002.0,DaCapo Press,http://images.amazon.com/images/P/030681188X.0...,http://images.amazon.com/images/P/030681188X.0...,http://images.amazon.com/images/P/030681188X.0...,How to Lose Friends &amp; Alienate PeopleToby ...
12304,0306812274,How to Lose Friends &amp; Alienate People,Toby Young,2003.0,Da Capo Press,http://images.amazon.com/images/P/0306812274.0...,http://images.amazon.com/images/P/0306812274.0...,http://images.amazon.com/images/P/0306812274.0...,How to Lose Friends &amp; Alienate PeopleToby ...
11885,0875420133,How to See and Read the Aura (Llewellyn's How ...,Ted Andrews,1991.0,Llewellyn Publications,http://images.amazon.com/images/P/0875420133.0...,http://images.amazon.com/images/P/0875420133.0...,http://images.amazon.com/images/P/0875420133.0...,How to See and Read the Aura (Llewellyn's How ...


---